# Getting the modules

In [35]:
# -*- coding: utf-8 -*-
"""
Created on Wed Feb  7 11:24:13 2024

@author: mrsag
"""

# import yt
import numpy as np
import matplotlib.pyplot as plt
# import glob
# from Curve_fitting_with_scipy import Gaussianfitting as Gf
from scipy.signal import fftconvolve
# from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import matplotlib.image as mpimg
from matplotlib.legend_handler import HandlerBase
from matplotlib.lines import Line2D
from matplotlib.path import Path
from skimage import io
from skimage.transform import resize
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from copy import deepcopy
from tqdm import tqdm
from mayavi import mlab

import matplotlib as mpl


mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'
mpl.rcParams['font.size'] = 12
#mpl.rcParams['font.weight'] = 'bold'
#mpl.rcParams['font.style'] = 'italic'  # Set this to 'italic'
mpl.rcParams['figure.dpi']=100 # highres display

# Basic functions:

In [36]:

def find_index(array, value):
    # Calculate the absolute differences between each element and the target value
    absolute_diff = np.abs(array - value)
    
    # Find the index of the minimum absolute difference
    index = np.argmin(absolute_diff)
    
    return index

def moving_average(signal, window_size):
    # Define the window coefficients for the moving average
    window = np.ones(window_size) / float(window_size)
    
    # Apply the moving average filter using fftconvolve
    filtered_signal = fftconvolve(signal, window, mode='same')
    
    return filtered_signal


# Defining the main classes and functions

In [ ]:
class SimulationBox3D:
    def __init__(self, resolution_x=50, resolution_y=50, resolution_z=50,
                 box_x=1.0, box_y=1.0, box_z=1.0, potential_offset=0):
        self.resolution_x = resolution_x
        self.resolution_y = resolution_y
        self.resolution_z = resolution_z

        self.box_x = box_x
        self.box_y = box_y
        self.box_z = box_z

        self.dx = np.diff(np.linspace(0,box_x,resolution_x))[0]
        self.dy = np.diff(np.linspace(0,box_y,resolution_y))[0]
        self.dz = np.diff(np.linspace(0,box_z,resolution_z))[0]

        # Create a proper Cartesian grid
        self.x, self.y, self.z = np.meshgrid(
            np.linspace(0, box_x, resolution_x),
            np.linspace(0, box_y, resolution_y),
            np.linspace(0, box_z, resolution_z),
            indexing='ij'
        )
        self.solved = False
        self.potential = np.full_like(self.x, potential_offset, dtype=float)
        self.fixed_mask = np.zeros_like(self.potential, dtype=bool)

    def add_sphere(self, center, radius, potential=0):
        cx, cy, cz = center
        mask = ((self.x - cx)**2 + (self.y - cy)**2 + (self.z - cz)**2) <= radius**2
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_box(self, x_bounds, y_bounds, z_bounds, potential=0):
        x0, x1 = x_bounds
        y0, y1 = y_bounds
        z0, z1 = z_bounds
        mask = ((self.x >= x0) & (self.x <= x1) &
                (self.y >= y0) & (self.y <= y1) &
                (self.z >= z0) & (self.z <= z1))
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_cylinder(self, base_center, radius, height, axis='z', potential=0):
        cx, cy, cz = base_center
        if axis == 'z':
            mask = (((self.x - cx)**2 + (self.y - cy)**2 <= radius**2) &
                    (self.z >= cz) & (self.z <= cz + height))
        elif axis == 'x':
            mask = (((self.y - cy)**2 + (self.z - cz)**2 <= radius**2) &
                    (self.x >= cx) & (self.x <= cx + height))
        elif axis == 'y':
            mask = (((self.x - cx)**2 + (self.z - cz)**2 <= radius**2) &
                    (self.y >= cy) & (self.y <= cy + height))
        else:
            raise ValueError("axis must be 'x', 'y', or 'z'")
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_hollow_pipe(self, base_center, radius, thickness, height, axis='z', potential=0):
        cx, cy, cz = base_center
        if axis == 'z':
            mask = (((self.x - cx)**2 + (self.y - cy)**2 <= (radius+thickness/2)**2) &
                    ((self.x - cx)**2 + (self.y - cy)**2 >= (radius-thickness/2)**2)
                    (self.z >= cz) & (self.z <= cz + height))
        elif axis == 'x':
            mask = (((self.y - cy)**2 + (self.z - cz)**2 <= (radius+thickness/2)**2) &
                    ((self.y - cy)**2 + (self.z - cz)**2 >= (radius-thickness/2)**2) &
                    (self.x >= cx) & (self.x <= cx + height))
        elif axis == 'y':
            mask = (((self.x - cx)**2 + (self.z - cz)**2 <= (radius+thickness/2)**2) &
                    ((self.x - cx)**2 + (self.z - cz)**2 >= (radius-thickness/2)**2)
                    (self.y >= cy) & (self.y <= cy + height))
        else:
            raise ValueError("axis must be 'x', 'y', or 'z'")
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_ellipsoid(self, center, radii, potential=0):
        cx, cy, cz = center
        rx, ry, rz = radii
        mask = (((self.x - cx)/rx)**2 + ((self.y - cy)/ry)**2 + ((self.z - cz)/rz)**2 <= 1)
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_hyperboloid(self, center, coeffs, waist=1, axis="z", potential=0):
        cx, cy, cz = center
        a, b, c = coeffs
        if axis=="x":
            mask = (-((self.x - cx)/a)**2 + ((self.y - cy)/b)**2 + ((self.z - cz)/c)**2 <= waist**2)
        if axis=="y":
            mask = (((self.x - cx)/a)**2 - ((self.y - cy)/b)**2 + ((self.z - cz)/c)**2 <= waist**2)
        if axis=="z":
            mask = (((self.x - cx)/a)**2 + ((self.y - cy)/b)**2 - ((self.z - cz)/c)**2 <= waist**2)

        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def add_plane(self, coefficients, thickness=0.01, potential=0):
        A, B, C, D = coefficients
        norm = np.sqrt(A**2 + B**2 + C**2)
        distance = (A * self.x + B * self.y + C * self.z + D) / norm
        mask = np.abs(distance) <= thickness / 2
        self.potential[mask] = potential
        self.fixed_mask[mask] = True

    def solve(self, max_iter=1000, tol=1e-4, method='jacobi', verbose=False):
        V = self.potential.copy()
        self.solved=True
        with tqdm(total=max_iter, desc="Solver Iteration", colour='green', ncols=100, dynamic_ncols=False) as pbar:
            for it in range(max_iter):
                V_old = V.copy()
                
                if method == 'jacobi':
                    V_new = V.copy()
                    V_new[1:-1, 1:-1, 1:-1] = 1/6 * (
                        V[2:, 1:-1, 1:-1] + V[:-2, 1:-1, 1:-1] +
                        V[1:-1, 2:, 1:-1] + V[1:-1, :-2, 1:-1] +
                        V[1:-1, 1:-1, 2:] + V[1:-1, 1:-1, :-2]
                    )
                    mask = ~self.fixed_mask[1:-1, 1:-1, 1:-1]
                    V[1:-1, 1:-1, 1:-1][mask] = V_new[1:-1, 1:-1, 1:-1][mask]
                elif method == 'gauss-seidel':
                    for i in range(1, self.resolution_x - 1):
                        for j in range(1, self.resolution_y - 1):
                            for k in range(1, self.resolution_z - 1):
                                if not self.fixed_mask[i, j, k]:
                                    V[i, j, k] = 1/6 * (V[i+1, j, k] + V[i-1, j, k] +
                                                    V[i, j+1, k] + V[i, j-1, k] +
                                                    V[i, j, k+1] + V[i, j, k-1])
                else:
                    raise ValueError("Unknown method")

                diff = np.abs(V - V_old).max()
                
                # Update progress bar
                pbar.set_postfix_str(f"Δ={diff:.2e}")
                pbar.update(1)

                if diff < tol:
                    if verbose:
                        tqdm.write(f"Converged at iteration {it}, max change: {diff:.2e}")
                    break
        
        self.potential = V
        tqdm.write(f"Final Max change in iteration {it+1}: {diff:.2e}")

    def plot_potential_isosurface(self,contours=50,opacity=0.4,cmap="jet"):
        if (not self.solved):
            self.solve(max_iter=300)
            
        nx, ny, nz = self.resolution_x, self.resolution_y, self.resolution_z
        lx, ly, lz = self.box_x, self.box_y, self.box_z

        # Create coordinate grid
        x, y, z = np.mgrid[
            0:lx:nx*1j,
            0:ly:ny*1j,
            0:lz:nz*1j
        ]

        mlab.figure(bgcolor=(1, 1, 1), size=(800, 600))
        mlab.contour3d(x, y, z, self.potential, contours=contours, opacity=opacity, colormap=cmap)
        axes = mlab.axes(xlabel='X', ylabel='Y', zlabel='Z',color=(1.0,0.0,0.0))
        mlab.colorbar(title="potential", orientation='vertical')
        axes.title_text_property.color = (1.0, 0.0, 0.0)  # red title text (if titles used)
        axes.label_text_property.color = (1.0, 0.0, 0.0)  # blue label text
        mlab.title("3D Isosurface of Potential")
        mlab.show()

    def plot_potential_density(self):
        if (not self.solved):
            self.solve(max_iter=300)
        # Create physical axes
        x = np.linspace(0, self.self_x, self.resolution_x)
        y = np.linspace(0, self.self_y, self.resolution_y)
        z = np.linspace(0, self.self_z, self.resolution_z)
        x, y, z = np.meshgrid(x, y, z, indexing='ij')  # Match shape with potential

        # Setup the scalar field for volumetric rendering
        src = mlab.pipeline.scalar_field(x, y, z, self.potential)

        # Optional: Adjust vmin/vmax to emphasize low potential regions (e.g., near zero)
        vmin = np.min(self.potential)
        vmax = np.max(self.potential)
        mlab.pipeline.volume(src, vmin=vmin, vmax=vmax)
        # Add axis, title, colorbar
        mlab.axes(xlabel='X', ylabel='Y', zlabel='Z')
        mlab.colorbar(title='Potential', orientation='vertical')
        mlab.title('3D Volume Rendering of Potential')
        mlab.view(azimuth=45, elevation=60, distance='auto')

        mlab.show()


    def electric_field(self):
        if(not self.solved):
            self.solve(max_iter=1000)
        
        self.Ex = -np.diff(self.potential,axis=0)/self.dx
        self.Ey = -np.diff(self.potential,axis=1)/self.dy
        self.Ez = -np.diff(self.potential,axis=2)/self.dz



    def plot_electric_field_3d(self, stepx=5, stepy=5, stepz=5, scale_factor=1.0,
                            remove_singularity=0, cut_edge=False, color_by_magnitude=True):
        self.electric_field()

        # Align Ex, Ey, Ez to a common grid: (nx-1, ny-1, nz-1)
        self.Ex = self.Ex[:, :-1, :-1]
        self.Ey = self.Ey[:-1, :, :-1]
        self.Ez = self.Ez[:-1, :-1, :]

          # Magnitude
        magnitude = np.sqrt(self.Ex**2 + self.Ey**2 + self.Ez**2)

        # Remove singularities (large spikes)
        if remove_singularity > 0:
            for _ in range(remove_singularity):
                idx = np.argmax(magnitude)
                i, j, k = np.unravel_index(idx, magnitude.shape)
                self.Ex[i, j, k] = 0
                self.Ey[i, j, k] = 0
                self.Ez[i, j, k] = 0
                magnitude[i, j, k] = 0

        # Aligned coordinates
        X = self.x[:-1, :-1, :-1][::stepx, ::stepy, ::stepz]
        Y = self.y[:-1, :-1, :-1][::stepx, ::stepy, ::stepz]
        Z = self.z[:-1, :-1, :-1][::stepx, ::stepy, ::stepz]
        
        
        # Downsample fields
        self.Ex = self.Ex[::stepx, ::stepy, ::stepz]
        self.Ey = self.Ey[::stepx, ::stepy, ::stepz]
        self.Ez = self.Ez[::stepx, ::stepy, ::stepz]

        # print(X.shape)
        # print(Y.shape)
        # print(Z.shape)

        # print(self.Ex.shape)
        # print(self.Ey.shape)
        # print(self.Ez.shape)

        # Scalars for coloring
        # scalars = magnitude if color_by_magnitude else None

        # Plot with Mayavi
        mlab.figure(bgcolor=(1, 1, 1))
        mlab.quiver3d(X, Y, Z, self.Ex, self.Ey, self.Ez,
                    mode='arrow',
                    colormap='jet')
        axes = mlab.axes(xlabel='X', ylabel='Y', zlabel='Z',color=(1.0,0.0,0.0))
        axes.title_text_property.color = (1.0, 0.0, 0.0)  # red title text (if titles used)
        axes.label_text_property.color = (1.0, 0.0, 0.0)  # blue label text
        if color_by_magnitude:
            mlab.colorbar(title='|E|', orientation='vertical')
        mlab.title('3D Electric Field')
        mlab.show()

        self.electric_field()




# Simulation for a case

In [38]:
# Create simulation box and add all geometries
box = SimulationBox3D(resolution_x=100,resolution_y=50,resolution_z=50,    # resolutions of grid along corresponding axis
                      box_x=2, box_y=1, box_z=1,    # Box side lengths 
                      potential_offset = 0)

# box.add_sphere(center=(0.25, 0.25, 0.25), radius=0.1, potential=1)
# box.add_box((0.6, 0.8), (0.6, 0.8), (0.6, 0.8), potential=-1)
# box.add_cylinder(base_center=(0.5, 0.5), radius=0.1, height=0.9, axis='z', potential=0.5)
# box.add_ellipsoid(center=(0.75, 0.25, 0.25), radii=(0.05, 0.1, 0.1), potential=0.8)
# box.add_hyperboloid(center=(1,0.2,0.2), coeffs=(0.6, 0.1, 0.1), waist=0.2, axis="x", potential=1)
# box.add_hyperboloid(center=(1,0.8,0.8), coeffs=(0.6, 0.1, 0.1), waist=0.2, axis="x", potential=-1)
# box.add_hyperboloid(center=(0.5,0.5,0.5), coeffs=(0.1, 0.1, 0.4), waist=0.2, axis="z", potential=-1)
# box.add_hyperboloid(center=(1.5,0.5,0.5), coeffs=(0.1, 0.4, 0.1), waist=0.2, axis="y", potential=1)
# box.add_plane(coefficients=(0,0,1,0),thickness=0.1,potential=10)
# box.add_plane(coefficients=(0,0,1,-1),thickness=0.1,potential=-10)
box.add_sphere(center=(0.5, 0.5, 0.5), radius=0.3, potential=10)
box.add_sphere(center=(1.5, 0.5, 0.5), radius=0.3, potential=-10)
# box.add_cylinder(base_center=(1,0.5,0),radius=0.1,axis="z",potential=1,height=1)
# box.add_cylinder(base_center=(0,0,0),radius=0.1,axis="x",potential=-5,height=2)
# box.add_cylinder(base_center=(0,1,0),radius=0.1,axis="x",potential=5,height=2)
# box.add_cylinder(base_center=(0,0,1),radius=0.1,axis="x",potential=5,height=2)
# box.add_cylinder(base_center=(0,1,1),radius=0.1,axis="x",potential=-5,height=2)
# box.add_hollow_pipe(base_center=(0,0.5,0.5),radius=0.1,axis="x",height=2, thickness=0.05,potential=5)



# Solve potential field
box.solve(max_iter=100, tol=1e-4, method='gauss-seidel', verbose=True)
box.electric_field()



Solver Iteration:   0%|                                                     | 0/100 [00:00<?, ?it/s]

Solver Iteration: 100%|███████████████████████████████| 100/100 [00:21<00:00,  4.71it/s, Δ=1.67e-02]

Final Max change in iteration 100: 1.67e-02


### 2D slice of one plane of simulation box

In [39]:
box.plot_electric_field_3d(stepx=2,stepy=2,stepz=2)

In [40]:
# Visualize potential in 3D (central slice)
# z_slice = 25
# fig = plt.figure(figsize=(8, 6))
# ax = fig.add_subplot(111, projection='3d')
# X, Y = np.meshgrid(np.linspace(0, 1, box.resolution_x),
#                    np.linspace(0, 1, box.resolution_y))
# Z = box.potential[:, :, z_slice]
# ax.plot_surface(X, Y, Z, cmap='viridis')
# ax.set_title('3D Potential Field (Z-Slice)')
# plt.tight_layout()
# plt.show()

### Full 3d isosurface view of potential with plotly

In [41]:
# # Normalize potential values for better visualization (optional)
# potential = box.potential


# # Define custom colorscale mimicking "jet" colormap
# colors = [
#     [0, "rgb(0,0,128)"],   # Dark blue
#     [0.25, "rgb(0,0,255)"], # Blue
#     [0.5, "rgb(0,255,255)"], # Cyan
#     [0.75, "rgb(255,255,0)"], # Yellow
#     [1, "rgb(255,0,0)"]   # Red
# ]

# # Flattened coordinates across actual physical dimensions
# x_vals = np.repeat(np.linspace(0, box.box_x, box.resolution_x), box.resolution_y * box.resolution_z)
# y_vals = np.tile(np.repeat(np.linspace(0, box.box_y, box.resolution_y), box.resolution_z), box.resolution_x)
# z_vals = np.tile(np.linspace(0, box.box_z, box.resolution_z), box.resolution_x * box.resolution_y)

# fig = go.Figure(data=go.Volume(
#     x=x_vals,
#     y=y_vals,
#     z=z_vals,
#     value=box.potential.flatten(),
#     opacity=0.2,
#     surface_count=20,
#     colorscale="plasma"
# ))


# fig.update_layout(
#     scene=dict(
#         xaxis_title='X',
#         yaxis_title='Y',
#         zaxis_title='Z'
#     ),
#     title='3D Potential Field Density Plot',
#     margin=dict(l=0, r=0, b=0, t=30)
# )

# fig.show(renderer="browser")

### 3D isosurface view with Mayavi

In [42]:
# from mayavi import mlab 

# # Assuming these attributes exist in your box object
# potential = box.potential
# nx, ny, nz = box.resolution_x, box.resolution_y, box.resolution_z
# lx, ly, lz = box.box_x, box.box_y, box.box_z

# # Create coordinate grid
# x, y, z = np.mgrid[
#     0:lx:nx*1j,
#     0:ly:ny*1j,
#     0:lz:nz*1j
# ]

# # Set custom contour levels including values near zero
# vmin = np.min(potential)
# vmax = np.max(potential)

# # Include zero-centered contours manually
# contours = np.linspace(vmin, vmax, 50)  # Or choose fewer/more levels depending on detail

# mlab.figure(bgcolor=(1, 1, 1), size=(800, 600))
# mlab.contour3d(x, y, z, potential, contours=50, opacity=0.4, colormap='jet')
# mlab.axes(xlabel='X', ylabel='Y', zlabel='Z')
# mlab.colorbar(title="Potential", orientation='vertical')
# mlab.title("3D Isosurface of Potential")
# mlab.show()


### Volumetric 3D plot with Mayavi

In [43]:
# import numpy as np
# from mayavi import mlab

# # Assume box.potential is a 3D NumPy array of shape (Nx, Ny, Nz)
# potential = box.potential

# # Create physical axes
# x = np.linspace(0, box.box_x, box.resolution_x)
# y = np.linspace(0, box.box_y, box.resolution_y)
# z = np.linspace(0, box.box_z, box.resolution_z)
# x, y, z = np.meshgrid(x, y, z, indexing='ij')  # Match shape with potential

# # Setup the scalar field for volumetric rendering
# src = mlab.pipeline.scalar_field(x, y, z, potential)

# # Optional: Adjust vmin/vmax to emphasize low potential regions (e.g., near zero)
# vmin = np.min(potential)
# vmax = np.max(potential)
# vol = mlab.pipeline.volume(src, vmin=vmin, vmax=vmax)

# # Add axis, title, colorbar
# mlab.axes(xlabel='X', ylabel='Y', zlabel='Z')
# mlab.colorbar(title='Potential', orientation='vertical')
# mlab.title('3D Volume Rendering of Potential')
# mlab.view(azimuth=45, elevation=60, distance='auto')

# mlab.show()


In [44]:


# src = mlab.pipeline.scalar_scatter(x, y, z, potential)

# # Connect them
# # src.mlab_source.dataset.lines = connections
# # src.update()

# # The stripper filter cleans up connected lines
# lines = mlab.pipeline.stripper(src)

# # Finally, display the set of lines
# mlab.pipeline.surface(lines, colormap='Accent', line_width=200, opacity=1)

# # And choose a nice view
# mlab.view(33.6, 106, 5.5, [0, 0, .05])
# mlab.roll(125)
# mlab.show()